# Lab 5 : Visual Servo

The goal for the visual servo lab is for the robot to steer toward a target object (in this case, a green cone).  We will do our initial testing **with the car on a block** before running the car on the ground.

1. Using the techniques from last session, we will use the camera to detect the center of a cone.
2. We will write a basic feedback controller, called a _bang-bang_ controller, which takes as input the coordinates of the center of the cone and produces a steering angle as output.
3. We will test out how this controller does when we deploy it on the robot.
4. Noticing some problems with the _bang bang_ style of control, we will implement a more sophisticated kind of controller, called a _proportional_ controller.
5. We will test this new controller out on the robot, and (hopefully) see that it performs better than the previous one.  You may need to tune the gain value _k_ on the proportional controller to improve performance.

Color prompts:
* **<font style="color:blue">Blue</font>** text indicates code to run without changing.
* **<font style="color:red">Red</font>** text indicates that there is a TODO section that you must complete in the code before running.
* **<font style="color:green">Green</font>** text indicates questions that you should answer before continuing.

Running:
* As with all labs where we want to run the motors, you will need to run `bash` and `teleop` in a Terminal first.
* To run individual cells, use control+enter.
* To reset everything again from the beginning, you can use Kernel > Restart and Clear Output.
* To run everything, you can use Kernel > Restart and Run All.  If you use this option, make sure that you get the initialization messages again from the first two cells.

**Important Note:
Read the instructions carefully to know which Part to run next.  You will initialize the racecar (Part 1), code the bang-bang controller (Part 2), then skip down to test it (Part 3 is at the bottom!), before coming back to code the proportional controller (Part 4) and test it (Part 3 again).**

## Part 1: Initialization

The code in the following cell imports several python libraries and a utility file (denoted with "utils" in the name).  When we "import" code, it allows us to use code stored in different files.  This keeps our lab file shorter and easier to read while also giving us the power of all the code that is already written in the imported files.

**<font style="color:blue">Run the following cell and wait for it to output the messages indicating success initializing ROS, racecar, and matrix diplay.</font>**
If you get a camera error, first, make sure teleop is running without an error. If that doesn't fix the issue, try unplugging the USB hub, waiting a few seconds, replugging the USB hub, and running again.  Restart teleop and the kernel whenever you are debugging hardware.

In [ ]:
# We will not be using simulation here, so this variable should never change
isSimulation = False

# Import Python libraries
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from nptyping import NDArray
from typing import Any, Tuple, List, Optional

# display tools
import IPython
import IPython.display
from io import BytesIO
import PIL.Image


# Import Racecar library
import sys
sys.path.insert(1, "../../library")
import racecar_core
import racecar_utils as rc_utils

**<font style="color:blue">Run the following cell.</font>**

In [ ]:
# Create global variables
display = None

### Perception helpers

The next block of code defines a function called "perception".  This function takes in the lower and upper HSV bounds of the color you want to detect.  It returns the center point of the detected color blob and the processed image.  Many of these steps were introduced in the last session.  The perception function runs other functions such as "get_mask" and "find contours".

**<font style="color:blue">Please run this block of code WITHOUT MAKING CHANGES to tell the program about the perception function.</font>**  This cell does not produce output.

In [ ]:
def get_mask(image, hsv_lower, hsv_upper):
    """   
    Returns a mask containing all of the areas of image which were between hsv_lower and hsv_upper.
    
    Args:
        image: The image (stored in BGR) from which to create a mask.
        hsv_lower: The lower bound of HSV values to include in the mask.
        hsv_upper: The upper bound of HSV values to include in the mask.
    """
    # Convert hsv_lower and hsv_upper to numpy arrays so they can be used by OpenCV
    hsv_lower = np.array(hsv_lower)
    hsv_upper = np.array(hsv_upper)
    
    image = cv.cvtColor(image, cv.COLOR_RGB2HSV)
    mask = cv.inRange(image, hsv_lower, hsv_upper)
    
    return mask

def perception(hsv_lower, hsv_upper):
    # Get raw image
    raw_image = rc.camera.get_color_image()

    # Process image

    # Step 1: HSV Segment
    mask = get_mask(raw_image, hsv_lower, hsv_upper)

    # Step 2: Compute contours
    contours, _ = cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    largest_contour = rc_utils.get_largest_contour(contours, min_area=30)

    # Step 3: Compute center
    if largest_contour is not None:
        center = rc_utils.get_contour_center(largest_contour)
    else:
        center = None

    # For visualization: Draw contour and center on image, if there is a cone
    processed_image = np.copy(raw_image)
    if largest_contour is not None:
        rc_utils.draw_contour(processed_image, largest_contour)
        rc_utils.draw_circle(processed_image, center)

    return center, processed_image

## Part 2: Bang-Bang Controller

### Preparing to Code the Bang-Bang Controller

Let's answer some questions to make sure we understand what we are trying to code before we begin coding.

**<font style="color:green">Write your answers to the following questions on a chalkboard.  Check with another group or a staff member when you're done BEFORE moving on to the coding exercises.  Ask that group or staff member to sign off on your chalkboard.</font>**

We will be using the camera to take an image of the scene ahead of the racecar.  Say we are looking for a green cone as our target object.  Once we set the correct HSV values to identify the color green, the perception code above will allow us to find the location of the center of the green cone in the image.  Those coordinates will be in pixels, in the format (row, column) counting from the top left corner.

Recall that the image captured by the racecar is 480 pixels tall by 640 pixels wide.

1.  If we want to turn the racecar toward a green cone, which coordinate value is more important, the vertical coordinate value (the row number) or the horizontal coordinate value (the column number)?

2. What is the column number at the horizontal center of the image?

3. If the green cone is centered horizontally in the image, what column number should be reported by the perception function as the (horizontal) center of the green cone?

4. If the cone is in the left half of the image, what range of values would be reported for the column of the cone center?

5. If the cone is in the right half of the image, what range of values would be reported for the column of the cone center?


Recall that when we are programming the robot, a turning angle of +1.0 is a full left turn while a turning angle of -1.0 is a full right turn.  We will use the maximum turn quantities (+1.0 and -1.0) when programming the bang-bang controller.


6. If the cone center's column number is 395, which way should the robot turn to face the cone?  What turning angle should be commanded?

7. If the cone center's column number is 5, which way should the robot turn to face the cone?  What turning angle should be commanded?

Make sure to check with another team or with a staff member before continuing on to the code!


### Bang-Bang Control helper

The next block of code is where we will implement a bang-bang controller to turn the racecar toward a target object.

**<font style="color:red">In the section marked TODO, implement an "if-else" block that sets the turn_angle according to the function's doc string.</font>**



In [ ]:
def bang_bang(center):
    ''' This function implements a bang-bang controller that turns
        the racecar toward a target in an image.  The function takes
        in a parameter called "center", which contains the row and
        column of the center of the target object in the racecar's
        camera view.

        First, the column number of the target object's center is
        extracted from the input "center" and named measured_column.

        Next, an "if" statement is used to compare the measured_column
        to the target column (i.e., the column number of the center
        of the image.)

        If the object is currently in the left half of the image,
        the turn_angle should be set to the full left-turn value.
        If the object is currently in the right half of the image,
        the turn_angle should be set to the full right-turn value.
    '''
    measured_column = center[1]

    ###### TODO: Implement bang bang control #####
    turn_angle = 0
    ##############################################

    return turn_angle


### Testing the Bang-Bang Controller
Let's test the bang-bang controller before writing the proportional controller.
Read these instructions, then skip down to Part 3. You will want to run the three Driving blocks after making sure that teleop is running.

While the racecar is **on a block**, hold down the right bumper [RB] to enable autonomous mode.  Move the green cone back and forth in front of the camera.  If everything is implemented correctly, you will see the wheels turn in the direction of the cone.

Call over a staff member.
**<font style="color:green">Answer the questions and have the staff member sign off on your whiteboard.</font>**

8. What do you observe about the steering behavior when the cone is far from center?
9. What do you observe about the steering behavior when the cone is near the center on either side?

If you would like, **get staff permission** to test the car on the floor.  Make sure there is lots of open space.  DO NOT do this facing a wall or obstacle.

After observing the behavior, move on to the Proportional controller.


## Part 4: Proportional Controller (skip to Part 3 first!)

### Preparing to Code the Proportional controller

**<font style="color:green">Answer the next questions on the board before working on the code.  Check in with another group or staff member, and ask them to sign off on your whiteboard.</font>**

10. If we are using a proportional controller, in which case should the car turn more sharply, when the center is at 395 or at 505?  Explain.

11. What is the error when the measured cone center is at 395?  (Remember, error is the desired value minus the measured value.)  Is this a positive or negative value?  Does the sign of your answer match the sign of your answer to #6?

We will probably need to test out different values for the proportional gain k to find an appropriate one for our racecar system.  A gain of k=0 means that the controller ignores the error and doesn't try to fix it at all.  If the gain value k is too large, the wheels will switch between maximum left steering and maximum right steering when the target gets too far from the center, similar to the bang-bang control.  Let's try to find an upper bound on our guesses for k.

12. We will have the maximum error if the cone is at the very edge of the image.  Say the cone center is at the very left edge of the image.  What is that maximum error in pixels?  What value of k would result in the maximum steering command of 1.0 for the maximum possible pixel error?

When you're trying out different k, make sure that the output turn angle you are commanding is in a reasonable range.


### Proportional Control helper

The next block of code is where we will implement a proportional controller to turn the racecar toward a target object.

**<font style="color:red">In the section marked TODO, implement the steps prompted by the function's doc string.</font>**



In [ ]:
def proportional(center, k):
    ''' This function implements a proportional controller that turns
        the racecar toward a target in an image.  The function takes
        in a parameter called "center", which contains the row and
        column of the center of the target object in the racecar's
        camera view.  The function also takes in a parameter k,
        which is the proportional gain.

        First, the column number of the target object's center is
        extracted from the input "center" and named measured_column.

        Second, the desired column number is defined.

        Third, the error is computed as the desired column minus
        the measured_column.

        Fourth, the turn_angle is computed by multiplying the
        column_error by the proportional gain k.
        (The value of k will be set as an input when the function
        is called--do not give k a value here.)
    '''
    measured_column = center[1]

    ##### TODO: Implement proportional control #####
    turn_angle = 0
    ################################################

    return turn_angle

### Testing the Proportional Controller
It's time to test the proportional controller.
You will need to change the function call in the second TODO in the Update Loop block so that the proportional controller is called instead of the bang-bang controller.  You can comment out the bang-bang command when you put in the proportional controller command so it's easy to switch between them.

While the racecar is **on a block**, hold down the right bumper [RB] to enable autonomous mode.  Move the green cone back and forth in front of the camera.  If everything is implemented correctly, you will see the wheels turn much more smoothly in the direction of the cone.

Call over a staff member.
**<font style="color:green">Answer the questions and have the staff member sign off on your whiteboard.</font>**
* What do you observe about the steering behavior when the cone is far from center? near the center on either side?
* How does the behavior of the proportional controller differ from the behavior of the bang-bang controller?

**Get staff permission** to test the car on the floor.  Make sure there is lots of open space.  DO NOT do this facing a wall or obstacle.

You may wish to tune the gain (i.e., try different values of k) to improve the performance.  With a good gain, the robot will turn smoothly toward the target without taking too long or oscillating too much.  Every time you change k, re-run the update block before re-running the run block.

## Part 3: Driving

### Update loop

The next code block defines the update() function that will run repeatedly while the robot is running.

First, PERCEPTION.
* The lower and upper HSV bounds of the target object are defined.  (Experienced python programmers will note that, technically, we could have defined these values outside of the loop to be more efficient, but we're keeping them here so everything is together.)
* The perception function captures an image and finds the center of the target object, if there is something that matches the color bounds.

Next, CONTROL.
* If a target object was detected, call the appropriate controller to get the steering angle.
* We have written the line "angle = bang_bang(center)" for you.  When you want to try out your proportional controller, you will need to instead write and run a line "angle = proportional(center, k=0.001)" to tell the code to run the proportional controller instead of the bang_bang controller.  You will be able to change the k value (the proportional gain) to see different behaviors with your proportional controller.
* The drive command is called with the calculated turn angle at the end of this section.

Last, VISUALIZATION.
* This section shows the camera image and can help make sure that the target object is being detected correctly. Showing the image takes a lot of computational power, though, and the output may be delayed.  If you don't need to see the image and the delay is causing problems, you can stop the display line from running by putting a # sign in front of the line "show_rgb_image_to_display(processed_image, rc, display)".  Adding the # symbol turns the line from a command for the program into a comment just to be read by humans.

Let's get to work!

**<font style="color:red">In the first TODO section, set the hsv_lower and hsv_upper bounds for a green cone.</font>** This is a REVIEW section from last session.  Don't spend too long on it.  Here are a few suggestions:
* Quickly figure good values out again
* Check the values in the **<font style="color:purple">Cone_Detection_MITES file</font>** to see what values people were using last week to see how well they work.
* Check in with another table that seems confident about their values :)
* Check in with course staff for a refresher on HSV values.

**<font style="color:red">In the second TODO section, make sure that the correct function is being called. The starter code already calls the bang-bang controller, so you will only need to change it when you want to run the proportional controller.</font>**




In [ ]:
# function to prepare display
def show_rgb_image_to_display(image, display):
    """
    Displays a color image in the Jupyter Notebook.
    Assumes image is in RGB format.
    """
    io = BytesIO()
    PIL.Image.fromarray(image).save(io, 'jpeg')
    img_display = IPython.display.Image(data=io.getvalue())
    display.update(img_display)

In [ ]:
display = IPython.display.display('', display_id=1)

### Run

Make sure that the car is **on the block** unless you have staff permission to run on the floor.

**<font style="color:blue">Run the next block to actually run the racecar.</font>**   The next block will enable autonomous mode when your press the right bumper [RB].  

Teleop needs to be running too, in order for the wheels to move.

In [ ]:
# Create Racecar
rc = racecar_core.create_racecar(isSimulation)

# Global varibles
global image_to_show

# [FUNCTION] The start function is run once every time the start button is pressed
def start():
    # Begin at a full stop
    rc.drive.stop()

    # Set update rate to 0.1 (Hz)
    rc.set_update_slow_time(0.1)

def update():
    '''
    This function runs repeatedly while the car is running in autonomous mode.
    The PERCEPTION section captures an image and finds the center of the
        target object, if present.
    The CONTROL section computes the correct control for the car and
        commands the car to drive with that control.
    The VISUALIZATION section shows the captured image.  It can be commented
        out if it is causing the program to run slowly.
    '''
    global image_to_show
    
    #-------- PERCEPTION --------

    ### TODO: REVIEW: Find HSV values ###
    hsv_lower = (0, 0, 0)
    hsv_upper = (179, 255, 255)
    ### END TODO: REVIEW: Find HSV values###

    # Capture an image and find the center of the target object, if present
    center, processed_image = perception(hsv_lower, hsv_upper)
    image_to_show = processed_image
    # ---------------------------

    #---------- CONTROL ---------
    if center is not None:
        # Drive towards the cone if there is one
        ### TODO: Run the correct controller ###
        angle = bang_bang(center)
        # angle = proportional(center, k=0.001)
        ### END TODO: Run the correct controller ###
        speed = 1.0
    else:
        # Otherwise, stop
        speed = 0.0
        angle = 0.0

    rc.drive.set_speed_angle(speed, angle)
    # ---------------------------

# Function that runs once every second
def update_slow():
    show_rgb_image_to_display(image_to_show, display)

########################################################################################
# DO NOT MODIFY: Register start and update and begin execution
########################################################################################

if __name__ == "__main__":
    rc.set_start_update(start, update, update_slow)
    rc.go()


## Completely Done?
If you are completely done early with both the bang-bang and proportional controllers, and testing on the ground, show a staff member!  You can then discuss project ideas, revisit an old lab, or try to visual servo toward a different colored object.  If you want to save your code from this lab for future use, you can download it onto the laptop and then email it to yourself.